In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
#!pip install accelerate
!pip install transformers==4.45.2
#!pip install bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy2
!pip install peft
#!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score

In [4]:
!git clone https://github.com/RefalMachine/llmtf_open
%cd llmtf_open
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl

Cloning into 'llmtf_open'...
remote: Enumerating objects: 514, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 514 (delta 60), reused 59 (delta 29), pack-reused 420 (from 1)
Receiving objects: 100% (514/514), 1.19 MiB | 4.82 MiB/s, done.
Resolving deltas: 100% (345/345), done.
/content/llmtf_open
--2024-10-24 14:13:20--  https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1291979 (1.2M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]   1.23M  6.69MB/s    in 0.2s    

2024-10-24 14:13:20 (6.69 MB/s) - ‘train.jsonl’ saved [1291979/1291979]



In [5]:
!ls

conversation_configs  examples		run_evaluate_multinode_multigpu.py   train.jsonl
Dockerfile	      llmtf		run_evaluate_multinode_multigpu.sh
eval_grammar.py       README.md		run_evaluate_singlenode_multigpu.sh
evaluate_model.py     requirements.txt	todo.txt


In [6]:
import codecs
import json
import copy
from collections import OrderedDict, defaultdict
import numpy as np
from tqdm import tqdm
import os
from datasets import load_dataset, Dataset
from typing import Dict, List, Tuple
from llmtf.metrics import mean, metric_max_over_ground_truths, f1_macro_score
import transformers.data.metrics.squad_metrics as squad_metrics
import re
from llmtf.base import Task, SimpleFewShotHFTask, LLM
from difflib import SequenceMatcher
import pandas as pd
import string

class MultiQ(SimpleFewShotHFTask):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.method = 'generate'
        self.dataset_name = 'multiq'
        self._max_new_tokens = 64

    @classmethod
    def name(cls):
        return 'darumeru/MultiQ'

    def dataset_args(self) -> Dict:
        return {'path': 'RefalMachine/darumeru', 'name': self.dataset_name}

    def aggregation(self) -> Dict:
        return {"f1": mean, "em": mean}

    def evaluate(self, sample, y_pred) -> Dict:
        y_true = [answer["segment"] for answer in sample['outputs']]
        f1 = metric_max_over_ground_truths(squad_metrics.compute_f1, y_pred, y_true)
        em = metric_max_over_ground_truths(squad_metrics.compute_exact, y_pred, y_true)

        return {
            "f1": f1,
            "em": em,
        }
    def test_split_name(self) -> str:
        return 'test'

    def prompt_split_name(self) -> str:
        return 'prompt'

    def create_messages(self, sample, with_answer=None) -> List[Dict]:
        # ignoring with_answer because it's already taken into account in the darumeru dataset
        messages = sample['messages']
        inputs = sample['inputs']
        for m in messages:
            m['content'] = m['content'].format(**inputs)
        return messages

In [7]:
task = MultiQ()

In [8]:
from llmtf.model import ApiVLLMModel

api_base = 'http://89.169.128.106:5000' # mistralai/Mistral-Nemo-Instruct-2407
#api_base = 'http://89.169.128.106:5001' # Qwen/Qwen2.5-14B-Instruct
#api_base = 'http://89.169.128.106:5002' # RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4
model = ApiVLLMModel(api_base)
model.from_pretrained()

model.generation_config.max_new_tokens = 200
model.generation_config.repetition_penalty = 1.0
model.generation_config.do_sample = False
model.generation_config.temperature = 0.0
model.generation_config

INFO: 2024-10-24 14:13:43,590: llmtf.base.apivllmmodel: ATTENTION! Hosting vLLM server must have vllm 0.6.3+
INFO:llmtf.base.apivllmmodel:ATTENTION! Hosting vLLM server must have vllm 0.6.3+


GenerationConfig {
  "max_new_tokens": 200,
  "temperature": 0.0,
  "top_k": 40,
  "top_p": 0.9
}

In [9]:
from llmtf.evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_dataset(
    task=task,
    model=model,
    output_dir='./multiq',
    max_len=4000,
    few_shot_count=0,
    generation_config=None, # will use model.generation_config by default
    batch_size=4,
    max_sample_per_dataset=200
)

INFO: 2024-10-24 14:13:44,274: llmtf.base.apivllmmodel: Updated generation_config.eos_token_id: None
INFO:llmtf.base.apivllmmodel:Updated generation_config.eos_token_id: None
INFO: 2024-10-24 14:13:44,279: llmtf.base.apivllmmodel: Updated generation_config.stop_strings: None
INFO:llmtf.base.apivllmmodel:Updated generation_config.stop_strings: None
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.31k [00:00<?, ?B/s]

data/multiq/test.jsonl:   0%|          | 0.00/3.01M [00:00<?, ?B/s]

data/multiq/prompt.jsonl:   0%|          | 0.00/31.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1046 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

100%|██████████| 200/200 [00:00<00:00, 2716.03it/s]
INFO: 2024-10-24 14:13:51,624: llmtf.base.darumeru/MultiQ: Loading Dataset: 7.34s
INFO:llmtf.base.darumeru/MultiQ:Loading Dataset: 7.34s
100%|██████████| 50/50 [03:22<00:00,  4.06s/it]
INFO: 2024-10-24 14:17:14,424: llmtf.base.darumeru/MultiQ: Processing Dataset: 202.80s
INFO:llmtf.base.darumeru/MultiQ:Processing Dataset: 202.80s
INFO: 2024-10-24 14:17:14,426: llmtf.base.darumeru/MultiQ: Results for darumeru/MultiQ:
INFO:llmtf.base.darumeru/MultiQ:Results for darumeru/MultiQ:
INFO: 2024-10-24 14:17:14,431: llmtf.base.darumeru/MultiQ: {'f1': 0.44145793052514404, 'em': 0.32}
INFO:llmtf.base.darumeru/MultiQ:{'f1': 0.44145793052514404, 'em': 0.32}


In [10]:
!ls ./multiq/

darumeru_MultiQ.jsonl  darumeru_MultiQ_params.jsonl  darumeru_MultiQ_total.jsonl


In [11]:
!cat ./multiq/darumeru_MultiQ_total.jsonl

{
    "task_name": "darumeru/MultiQ",
    "results": {
        "f1": 0.44145793052514404,
        "em": 0.32
    },
    "leaderboard_result": 0.38072896526257205
}


In [12]:
!cat ./multiq/darumeru_MultiQ_params.jsonl

{
    "custom_generation_config": null,
    "model_params": {
        "model_name_or_path": "mistralai/Mistral-Nemo-Instruct-2407",
        "api_base": "http://89.169.128.106:5000",
        "generation_config": {
            "max_new_tokens": 200,
            "temperature": 0.0,
            "top_k": 40,
            "top_p": 0.9,
            "transformers_version": "4.45.2"
        },
        "max_model_len": null
    },
    "task_params": {
        "max_len": 4000,
        "few_shot_count": 0,
        "batch_size": 4,
        "max_sample_per_dataset": 200,
        "method": "generate"
    }
}
